In [1]:
from cipher_placeholder import original_message

In [2]:
from string import ascii_lowercase

In [3]:
from random import sample

In [4]:
from copy import deepcopy

In [5]:
class CharMapper(object):
    
    def __init__(self, char_map=None):
        if char_map is None:
            self._char_map = {}
            for k, v in zip(sample(ascii_lowercase, 26), ascii_lowercase):
                self._char_map[k] = v
        else:
            self._char_map = deepcopy(char_map)
        self._char_map['.'] = '.'
        self._char_map[','] = ','
    
    def __call__(self, msg):
        cipher = ''.join(
            self._char_map.get(c, ' ')
            for c in msg.strip().lower()
        )
        return cipher
    
    def __deepcopy__(self, memo):
        cls = type(self)
        return cls(self._char_map)
    
    @property
    def char_map(self):
        return self._char_map
    
    def mutate(self):
        char1, char2 = sample(ascii_lowercase, 2)
        v1 = self._char_map[char1]
        v2 = self._char_map[char2]
        new_char_map = deepcopy(self.char_map)
        new_char_map[char1] = v2
        new_char_map[char2] = v1
        return type(self)(new_char_map)

In [6]:
cipher = CharMapper()

In [7]:
original_message

'I then lounged down the street and found,\nas I expected, that there was a mews in a lane which runs down\nby one wall of the garden. I lent the ostlers a hand in rubbing\ndown their horses, and received in exchange twopence, a glass of\nhalf-and-half, two fills of shag tobacco, and as much information\nas I could desire about Miss Adler, to say nothing of half a dozen\nother people in the neighbourhood in whom I was not in the least\ninterested, but whose biographies I was compelled to listen to.\n'

In [8]:
encrypted_message = cipher(original_message)
encrypted_message

'k ljnm cysmdnu uygm ljn tlpnnl wmu xysmu, wt k neonhlnu, ljwl ljnpn gwt w ingt km w cwmn gjkhj psmt uygm fq ymn gwcc yx ljn dwpunm. k cnml ljn ytlcnpt w jwmu km psffkmd uygm ljnkp jyptnt, wmu pnhnkbnu km nehjwmdn lgyonmhn, w dcwtt yx jwcx wmu jwcx, lgy xkcct yx tjwd lyfwhhy, wmu wt ishj kmxypiwlkym wt k hyscu untkpn wfysl iktt wucnp, ly twq myljkmd yx jwcx w uyvnm yljnp onyocn km ljn mnkdjfyspjyyu km gjyi k gwt myl km ljn cnwtl kmlnpntlnu, fsl gjytn fkydpwojknt k gwt hyionccnu ly cktlnm ly.'

In [9]:
import requests
import os

In [10]:
if not os.path.exists('moby_dict.txt'):
    res = requests.get('https://lazyprogrammer.me/course_files/moby_dick.txt')
    with open('moby_dict.txt', 'w', encoding='utf8') as fid:
        fid.write(res.content.decode('utf8'))

In [11]:
with open('moby_dict.txt', 'r') as fid:
    content = fid.read().lower()

In [12]:
from collections import Counter

In [13]:
valid_chars = set(ascii_lowercase)

In [31]:
unigram_cnter = Counter()
bigram_cnter = Counter()
for i in range(len(content)):
    c = content[i]
    if c not in valid_chars:
        continue
    unigram_cnter[c] += 1
    bigram = ''.join(filter(lambda c: c in valid_chars, content[i:i+2]))
    if len(bigram) == 2:
        bigram_cnter[bigram] += 1
total_support = sum(unigram_cnter.values())

In [32]:
from math import log

In [33]:
def compute_likelihood(decrypted_message):
    msg = decrypted_message.replace(' ', '')
    log_likelihood = 0
    for i in range(0, len(msg)-2):
        cnt_bigram = bigram_cnter[msg[i:i+2]]
        cnt_unigram = unigram_cnter[msg[i]]
        prob = (1 + cnt_bigram) / (cnt_unigram + 26)
        log_likelihood += log(prob)
    return log_likelihood

In [34]:
from itertools import chain

In [35]:
def create_offspring(deciphers, num_offspring):
    return chain(
        *[[decipher.mutate() for _ in range(num_offspring)] for decipher in deciphers],
        deciphers,
    )

In [49]:
num_epoches = 1000

In [71]:
from random import seed

In [72]:
seed(1234)

In [73]:
decipers = [CharMapper() for _ in range(60)]
for i in range(1, num_epoches+1):
    if i > 1:
        decipers = create_offspring(decipers, 3)
    pairs = [
        (compute_likelihood(decipher(encrypted_message)), decipher)
        for decipher in decipers
    ]
    pairs = sorted(pairs, key=lambda p: p[0], reverse=True)
    if i % 100 == 0:
        print(pairs[0][0])
    decipers = [p[1] for p in pairs[:15]]

-1254.8969681016429
-1230.370004376716
-1230.370004376716
-1230.370004376716
-1230.370004376716
-1230.370004376716
-1230.370004376716
-1230.370004376716
-1230.370004376716
-1230.370004376716


In [74]:
best_decipher = decipers[0]

In [75]:
encrypted_message

'k ljnm cysmdnu uygm ljn tlpnnl wmu xysmu, wt k neonhlnu, ljwl ljnpn gwt w ingt km w cwmn gjkhj psmt uygm fq ymn gwcc yx ljn dwpunm. k cnml ljn ytlcnpt w jwmu km psffkmd uygm ljnkp jyptnt, wmu pnhnkbnu km nehjwmdn lgyonmhn, w dcwtt yx jwcx wmu jwcx, lgy xkcct yx tjwd lyfwhhy, wmu wt ishj kmxypiwlkym wt k hyscu untkpn wfysl iktt wucnp, ly twq myljkmd yx jwcx w uyvnm yljnp onyocn km ljn mnkdjfyspjyyu km gjyi k gwt myl km ljn cnwtl kmlnpntlnu, fsl gjytn fkydpwojknt k gwt hyionccnu ly cktlnm ly.'

In [76]:
best_decipher(encrypted_message)

'i then lounged down the street and found, as i expected, that there was a mews in a lane which runs down by one wall of the garden. i lent the ostlers a hand in rubbing down their horses, and received in exchange twopence, a glass of half and half, two fills of shag tobacco, and as much information as i could desire about miss adler, to say nothing of half a doken other people in the neighbourhood in whom i was not in the least interested, but whose biographies i was compelled to listen to.'

In [77]:
original_message.lower().replace('\n', ' ').replace('-', ' ').strip()

'i then lounged down the street and found, as i expected, that there was a mews in a lane which runs down by one wall of the garden. i lent the ostlers a hand in rubbing down their horses, and received in exchange twopence, a glass of half and half, two fills of shag tobacco, and as much information as i could desire about miss adler, to say nothing of half a dozen other people in the neighbourhood in whom i was not in the least interested, but whose biographies i was compelled to listen to.'

In [78]:
original_message

'I then lounged down the street and found,\nas I expected, that there was a mews in a lane which runs down\nby one wall of the garden. I lent the ostlers a hand in rubbing\ndown their horses, and received in exchange twopence, a glass of\nhalf-and-half, two fills of shag tobacco, and as much information\nas I could desire about Miss Adler, to say nothing of half a dozen\nother people in the neighbourhood in whom I was not in the least\ninterested, but whose biographies I was compelled to listen to.\n'